## Prophet: To predict residual

In [1]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import lightgbm as lgbm
from prophet import Prophet

In [2]:
import warnings
warnings.filterwarnings("ignore")

# To hide prophet training logging
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [3]:
sns.set_theme()
plt.rcParams["figure.figsize"] = (10,5)

In [4]:
def smape(y_pred, y):
    return np.mean(np.abs(y_pred - y) / (np.abs(y) + np.abs(y_pred)))

In [5]:
# Dataset class

class Dataset:
    def __init__(self, df, cat_cols, num_cols, target_col):
        self.df = df
        self.cat_cols = cat_cols
        self.cat_encoded_cols = [col + "_encoded" for col in cat_cols]
        self.num_cols = num_cols
        self.target_col = target_col

    def create_lgbm_dataset(self, train_valid_test):
        assert train_valid_test in ("TRAIN", "VALID", "TEST", "ALL")
        if train_valid_test == 'ALL':
            data = self.df.loc[:, self.cat_encoded_cols + self.num_cols]
            label = self.df.loc[:, self.target_col].rename()
        else:
            data = self.df.loc[self.df["train_valid_test"] == train_valid_test, self.cat_encoded_cols + self.num_cols]
            label = self.df.loc[self.df["train_valid_test"] == train_valid_test, self.target_col]

        return lgbm.Dataset(
            data = data,
            label = label,
            feature_name = self.cat_encoded_cols + self.num_cols,
            categorical_feature = self.cat_encoded_cols,
        )
    
    @staticmethod
    def create_prophet_dataframe(df, df_y_pred):
        df_y_pred = df_y_pred.rename(columns={"y":"trend", "y_pred":"trend_pred"})
        df = pd.concat([df, df_y_pred], axis=1)
        df["residual"] = df["sold"] - df["trend_pred"]

        # train valid test split
        df.loc[df.d<1914, 'train_valid_test'] = 'TRAIN'
        df.loc[(df.d>=1914) & (df.d<1942), 'train_valid_test'] = 'VALID'
        df.loc[df.d>=1942, 'train_valid_test'] = 'TEST'

        df_prophet = df.rename(columns={"date":"ds", "residual":"y"})
        return df_prophet

In [6]:
df = pd.read_pickle('./saved/data/preprocessed_df.pkl')
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,id_sold_lag0_ma7,id_sold_lag0_ma7_diff,id_sold_lag0_ma28,id_sold_lag0_ma28_diff,global_sold_lag0_ma7,global_sold_lag0_ma7_diff,global_sold_lag0_ma28,global_sold_lag0_ma28_diff,sold_lag1,avg_sold_per_id
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1799,0.0,2016-01-01,11548,...,0.428571,0.00000,0.785714,-0.107143,35064.142857,5797.628571,36218.607143,-197.714286,2.0,0.804196
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1800,0.0,2016-01-02,11549,...,0.285714,-0.17764,0.785714,0.000000,35972.714286,1129.788820,36136.892857,-81.714286,0.0,0.804196
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1801,0.0,2016-01-03,11549,...,0.285714,0.00000,0.785714,0.000000,37253.000000,1592.007453,35991.357143,-145.535714,0.0,0.804196
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1802,1.0,2016-01-04,11549,...,0.428571,0.17764,0.750000,-0.035714,38214.285714,1195.337888,36037.250000,45.892857,0.0,0.804196
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1803,0.0,2016-01-05,11549,...,0.428571,0.00000,0.607143,-0.142857,38701.857143,606.284472,36122.714286,85.464286,1.0,0.804196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5213272,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,2016-06-15,11620,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,6620.392857,-1324.857143,NaN,0.062937
5213273,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,2016-06-16,11620,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,5300.285714,-1320.107143,NaN,0.062937
5213274,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,2016-06-17,11620,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,3780.571429,-1519.714286,NaN,0.062937
5213275,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,2016-06-18,11621,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1940.642857,-1839.928571,NaN,0.062937


In [7]:
df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sold', 'date', 'wm_yr_wk', 'weekday', 'is_holiday', 'is_weekend',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'sell_price_diff',
       'id_sold_lag1_ma7', 'id_sold_lag1_ma7_diff', 'id_sold_lag1_ma28',
       'id_sold_lag1_ma28_diff', 'global_sold_lag1_ma7',
       'global_sold_lag1_ma7_diff', 'global_sold_lag1_ma28',
       'global_sold_lag1_ma28_diff', 'id_sold_lag0_ma7',
       'id_sold_lag0_ma7_diff', 'id_sold_lag0_ma28', 'id_sold_lag0_ma28_diff',
       'global_sold_lag0_ma7', 'global_sold_lag0_ma7_diff',
       'global_sold_lag0_ma28', 'global_sold_lag0_ma28_diff', 'sold_lag1',
       'avg_sold_per_id'],
      dtype='object')

In [8]:
target = 'residual'     # sold, id_sold_lag0_ma28, residual

# create target
def get_target(df, target):
    if target not in ("sold", "id_sold_lag0_ma28", "residual"):
        raise ValueError(
            "target must be 'sold' or 'id_sold_lag0_ma28' or 'residual'"
        )
    elif target=="residual":
        df_y_pred = pd.read_csv('./saved/data/df_y_pred.csv')   #y: id_sold_lag0_ma28
        if not np.allclose(df_y_pred["y"].values, df["id_sold_lag0_ma28"].values):
            raise ValueError(
                "y of df_y_pred.csv doesn't match id_sold_lag0_ma28 of df_left.csv"
            )
        df = Dataset.create_prophet_dataframe(df, df_y_pred)
    else:
        pass
    return df
        
df = get_target(df, target)
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,ds,wm_yr_wk,...,global_sold_lag0_ma7,global_sold_lag0_ma7_diff,global_sold_lag0_ma28,global_sold_lag0_ma28_diff,sold_lag1,avg_sold_per_id,trend,trend_pred,y,train_valid_test
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1799,0.0,2016-01-01,11548,...,35064.142857,5797.628571,36218.607143,-197.714286,2.0,0.804196,0.785714,0.89,-0.89,TRAIN
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1800,0.0,2016-01-02,11549,...,35972.714286,1129.788820,36136.892857,-81.714286,0.0,0.804196,0.785714,0.78,-0.78,TRAIN
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1801,0.0,2016-01-03,11549,...,37253.000000,1592.007453,35991.357143,-145.535714,0.0,0.804196,0.785714,0.78,-0.78,TRAIN
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1802,1.0,2016-01-04,11549,...,38214.285714,1195.337888,36037.250000,45.892857,0.0,0.804196,0.750000,0.79,0.21,TRAIN
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1803,0.0,2016-01-05,11549,...,38701.857143,606.284472,36122.714286,85.464286,1.0,0.804196,0.607143,0.75,-0.75,TRAIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5213272,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,2016-06-15,11620,...,0.000000,0.000000,6620.392857,-1324.857143,NaN,0.062937,0.000000,0.00,NaN,TEST
5213273,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,2016-06-16,11620,...,0.000000,0.000000,5300.285714,-1320.107143,NaN,0.062937,0.000000,0.00,NaN,TEST
5213274,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,2016-06-17,11620,...,0.000000,0.000000,3780.571429,-1519.714286,NaN,0.062937,0.000000,0.00,NaN,TEST
5213275,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,2016-06-18,11621,...,0.000000,0.000000,1940.642857,-1839.928571,NaN,0.062937,0.000000,0.00,NaN,TEST


In [9]:
df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sold', 'ds', 'wm_yr_wk', 'weekday', 'is_holiday', 'is_weekend',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'sell_price_diff',
       'id_sold_lag1_ma7', 'id_sold_lag1_ma7_diff', 'id_sold_lag1_ma28',
       'id_sold_lag1_ma28_diff', 'global_sold_lag1_ma7',
       'global_sold_lag1_ma7_diff', 'global_sold_lag1_ma28',
       'global_sold_lag1_ma28_diff', 'id_sold_lag0_ma7',
       'id_sold_lag0_ma7_diff', 'id_sold_lag0_ma28', 'id_sold_lag0_ma28_diff',
       'global_sold_lag0_ma7', 'global_sold_lag0_ma7_diff',
       'global_sold_lag0_ma28', 'global_sold_lag0_ma28_diff', 'sold_lag1',
       'avg_sold_per_id', 'trend', 'trend_pred', 'y', 'train_valid_test'],
      dtype='object')

In [10]:
df_calendar_trim = pd.read_csv('./saved/data/df_calendar_trim.csv')
df_holiday = pd.concat([
    df_calendar_trim.loc[df_calendar_trim.event_name_1.notnull(), ["date", "event_name_1"]].rename(columns={"date":"ds", "event_name_1":"holiday"}),
    df_calendar_trim.loc[df_calendar_trim.event_name_2.notnull(), ["date", "event_name_2"]].rename(columns={"date":"ds", "event_name_2":"holiday"})
], axis=0).reset_index(drop=True)

df_holiday

,ds,holiday
0,2015-12-14,Chanukah End
1,2015-12-25,Christmas
2,2016-01-01,NewYear
3,2016-01-07,OrthodoxChristmas
4,2016-01-18,MartinLutherKingDay
5,2016-02-07,SuperBowl
6,2016-02-10,LentStart
7,2016-02-14,ValentinesDay
8,2016-02-15,PresidentsDay
9,2016-02-17,LentWeek2


In [11]:
# grid search
from sklearn.model_selection import ParameterGrid

# param setting
PARAMS_GRID = {  
    "changepoint_range": [0.5, 0.6, 0.7, 0.8, 0.9],
}
PROPHET_PARAMS = {
    "yearly_seasonality": True,
    "weekly_seasonality": True,
    "daily_seasonality": False,
    "seasonality_mode": 'additive',
    "changepoint_prior_scale": 0.025,
    "seasonality_prior_scale": 1,
    "holidays_prior_scale": 0.02,
    #"changepoint_range": 0.8
}
REGRESSORS = ['is_holiday', 'is_weekend', 'snap_CA', 'snap_TX', 'snap_WI', "sell_price", "sell_price_diff"]
ROUND_DECIMAL = 2


# start
all_gs_results_list = []
grid = ParameterGrid(PARAMS_GRID)
print(f'Total Possible Models: {len(grid)}')

for n, params in enumerate(tqdm(grid)):
    print(f"Processing {n}: {params}")

    df_pred = pd.DataFrame()

    np.random.seed(1999)
    list_id = np.random.choice(df["id"].unique(),20, replace=False)

    for i, id in enumerate(list_id):
        # split by id
        df_per_id = df.loc[df['id'] == id]

        # Prophet setting
        model = Prophet(holidays=df_holiday, **PROPHET_PARAMS, **params)
        model.add_country_holidays(country_name='US')
        for regressor in REGRESSORS:
            model.add_regressor(regressor)

        # Prophet train
        model.fit(df_per_id.loc[df_per_id['train_valid_test']=="TRAIN"])

        # Prophet predict
        forecast = model.predict(df_per_id)

        # Rounding
        y_pred = forecast["yhat"].values
        if type(ROUND_DECIMAL) == int:
            y_pred = np.round(y_pred, ROUND_DECIMAL)
        df_per_id["y_pred"] = y_pred

        # Concat all id tgt
        df_pred = pd.concat([df_pred, df_per_id], axis=0)
    
    # residual-wise smape
    train_smape = smape(
        df_pred.loc[df_pred.train_valid_test == "TRAIN", "y_pred"],
        df_pred.loc[df_pred.train_valid_test == "TRAIN", "y"],
    )
    valid_smape = smape(
        df_pred.loc[df_pred.train_valid_test == "VALID", "y_pred"],
        df_pred.loc[df_pred.train_valid_test == "VALID", "y"],
    )

    gs_result = {"n": n, "params": params, "train_smape": train_smape, "valid_smape": valid_smape}
    all_gs_results_list.append(gs_result)

    print(f"Train smape: {train_smape*100:.2f}%")
    print(f"Valid smape: {valid_smape*100:.2f}%")

df_gs_result = pd.DataFrame(all_gs_results_list)
df_gs_result

Total Possible Models: 5


  0%|          | 0/5 [00:00<?, ?it/s]

Processing 0: {'changepoint_range': 0.5}


 20%|██        | 1/5 [00:24<01:36, 24.21s/it]

Train smape: 67.04%
Valid smape: 70.87%
Processing 1: {'changepoint_range': 0.6}


...

 80%|████████  | 4/5 [01:31<00:22, 22.74s/it]

Train smape: 67.48%
Valid smape: 70.76%
Processing 4: {'changepoint_range': 0.9}


100%|██████████| 5/5 [01:54<00:00, 22.84s/it]

Train smape: 66.93%
Valid smape: 70.94%


,n,params,train_smape,valid_smape
0,0,{'changepoint_range': 0.5},0.670389,0.708655
1,1,{'changepoint_range': 0.6},0.671761,0.709202
2,2,{'changepoint_range': 0.7},0.673954,0.709376
3,3,{'changepoint_range': 0.8},0.674789,0.707638
4,4,{'changepoint_range': 0.9},0.669302,0.709384


In [12]:
ROUND_DECIMAL = 2

PROPHET_PARAMS = {
    "yearly_seasonality": True,
    "weekly_seasonality": True,
    "daily_seasonality": False,
    "seasonality_mode": 'additive',
    "changepoint_prior_scale": 0.025,
    "seasonality_prior_scale": 1,
    "holidays_prior_scale": 0.02,
    "changepoint_range": 0.8
}

REGRESSORS = ['is_holiday', 'is_weekend', 'snap_CA', 'snap_TX', 'snap_WI', "sell_price", "sell_price_diff"]

df_pred = pd.DataFrame()

np.random.seed(1999)
# list_id = np.random.choice(df["id"].unique(),50, replace=False)
list_id = df["id"].unique()

for i, id in enumerate(tqdm(list_id)):
    # split by id
    df_per_id = df.loc[df['id'] == id]

    # Prophet setting
    model = Prophet(holidays=df_holiday, **PROPHET_PARAMS)
    model.add_country_holidays(country_name='US')
    for regressor in REGRESSORS:
        model.add_regressor(regressor)

    # Prophet train
    model.fit(df_per_id.loc[df_per_id['train_valid_test']=="TRAIN"])

    # Prophet predict
    forecast = model.predict(df_per_id)

    # Rounding
    y_pred = forecast["yhat"].values
    if type(ROUND_DECIMAL) == int:
        y_pred = np.round(y_pred, ROUND_DECIMAL)
    df_per_id["y_pred"] = y_pred

    # Concat all id tgt
    df_pred = pd.concat([df_pred, df_per_id], axis=0)
    # if i==1:
    #     break

100%|██████████| 30490/30490 [11:45:35<00:00,  1.39s/it]  


In [16]:
# residual-wise smape
train_smape = smape(
    df_pred.loc[df_pred.train_valid_test == "TRAIN", "y_pred"],
    df_pred.loc[df_pred.train_valid_test == "TRAIN", "y"],
)
valid_smape = smape(
    df_pred.loc[df_pred.train_valid_test == "VALID", "y_pred"],
    df_pred.loc[df_pred.train_valid_test == "VALID", "y"],
)

print(f"Train smape: {train_smape*100:.2f}%")
print(f"Valid smape: {valid_smape*100:.2f}%")

Train smape: 68.01%
Valid smape: 68.67%


In [17]:
ROUND_DECIMAL=0

# sold-wise smape
df_pred['sold_pred'] = df_pred['trend_pred'] + df_pred['y_pred']

# clip at 0 & round to int
df_pred['sold_pred'] = np.clip(df_pred['sold_pred'].values, a_min=0, a_max=None)
if type(ROUND_DECIMAL) == int:
    df_pred['sold_pred'] = np.round(df_pred['sold_pred'].values, ROUND_DECIMAL)

train_smape = smape(
    df_pred.loc[df_pred.train_valid_test == "TRAIN", "sold_pred"],
    df_pred.loc[df_pred.train_valid_test == "TRAIN", "sold"],
)
valid_smape = smape(
    df_pred.loc[df_pred.train_valid_test == "VALID", "sold_pred"],
    df_pred.loc[df_pred.train_valid_test == "VALID", "sold"],
)

print(f"Train smape: {train_smape*100:.2f}%")
print(f"Valid smape: {valid_smape*100:.2f}%")

Train smape: 52.78%
Valid smape: 65.27%


In [18]:
df_pred.to_csv("./saved/data/df_pred_residual.csv")

In [19]:
df_pred

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,ds,wm_yr_wk,...,global_sold_lag0_ma28,global_sold_lag0_ma28_diff,sold_lag1,avg_sold_per_id,trend,trend_pred,y,train_valid_test,y_pred,sold_pred
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1799,0.0,2016-01-01,11548,...,36218.607143,-197.714286,2.0,0.804196,0.785714,0.89,-0.89,TRAIN,-0.18,1.0
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1800,0.0,2016-01-02,11549,...,36136.892857,-81.714286,0.0,0.804196,0.785714,0.78,-0.78,TRAIN,-0.23,1.0
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1801,0.0,2016-01-03,11549,...,35991.357143,-145.535714,0.0,0.804196,0.785714,0.78,-0.78,TRAIN,-0.42,0.0
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1802,1.0,2016-01-04,11549,...,36037.250000,45.892857,0.0,0.804196,0.750000,0.79,0.21,TRAIN,0.30,1.0
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1803,0.0,2016-01-05,11549,...,36122.714286,85.464286,1.0,0.804196,0.607143,0.75,-0.75,TRAIN,-0.43,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5213272,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,2016-06-15,11620,...,6620.392857,-1324.857143,NaN,0.062937,0.000000,0.00,NaN,TEST,0.06,0.0
5213273,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,2016-06-16,11620,...,5300.285714,-1320.107143,NaN,0.062937,0.000000,0.00,NaN,TEST,0.11,0.0
5213274,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,2016-06-17,11620,...,3780.571429,-1519.714286,NaN,0.062937,0.000000,0.00,NaN,TEST,0.05,0.0
5213275,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,2016-06-18,11621,...,1940.642857,-1839.928571,NaN,0.062937,0.000000,0.00,NaN,TEST,-0.01,0.0
